Building a Simple LLM Application with LCEL

This application will translate text from English into another language. 



In [1]:
!pip install langchain


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
## Open AI API Key and Open Source models--Llama3,Gemma2,mistral--Groq
from dotenv import load_dotenv
load_dotenv()

GROQ_API_KEY="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"

import os

groq_api_key = os.getenv("GROQ_API_KEY")
groq_api_key


In [7]:
!pip install langchain_groq


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
from langchain_groq import ChatGroq

groq_api_key = "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"  
model = ChatGroq(model="Gemma2-9b-It", groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000177A5525910>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000177A54DD310>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [11]:
!pip install langchain_core


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
from langchain_core.messages import HumanMessage,SystemMessage

messages=[
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello How are you?")
]

result=model.invoke(messages)


In [14]:
result

AIMessage(content='Here are a few ways to say "Hello, how are you?" in French:\n\n* **Bonjour, comment allez-vous ?** (Formal)\n* **Salut, comment vas-tu ?** (Informal, to one person)\n* **Salut, comment allez-vous ?** (Informal, to multiple people) \n\n\nLet me know if you\'d like to learn more French phrases!\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 86, 'prompt_tokens': 21, 'total_tokens': 107, 'completion_time': 0.156363636, 'prompt_time': 0.000166489, 'queue_time': 0.014191651, 'total_time': 0.156530125}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-d4823ffd-3818-4be8-9ee3-200260f0b6ec-0', usage_metadata={'input_tokens': 21, 'output_tokens': 86, 'total_tokens': 107})

In [15]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()
parser.invoke(result)

'Here are a few ways to say "Hello, how are you?" in French:\n\n* **Bonjour, comment allez-vous ?** (Formal)\n* **Salut, comment vas-tu ?** (Informal, to one person)\n* **Salut, comment allez-vous ?** (Informal, to multiple people) \n\n\nLet me know if you\'d like to learn more French phrases!\n'

In [16]:
 ### Using LCEL-chain the components
chain=model|parser
chain.invoke(messages)

'Here are a few ways to say "Hello, how are you?" in French:\n\n* **Bonjour, comment allez-vous ?** (Formal)\n* **Salut, ça va ?** (Informal)\n* **Coucou, comment vas-tu ?** (Very informal, used with friends or family)\n\n\nLet me know if you\'d like to learn other ways to greet someone in French!\n'

In [17]:
##Prompt Templates
from langchain_core.prompts import ChatPromptTemplate

generic_template="Translate the following into{language}:"

prompt=ChatPromptTemplate.from_messages(
    [("system",generic_template),("user","{text}")]
)



In [19]:
result=prompt.invoke({"language":"French","text":"hello"})

In [20]:
result.to_messages()

[SystemMessage(content='Translate the following intoFrench:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hello', additional_kwargs={}, response_metadata={})]

In [22]:
##Chaining together components with LCEL
chain=prompt|model|parser
chain.invoke({"language":"French","text":"Hello"})

'Bonjour \n'

In [23]:
!pip install fastapi


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached fastapi-0.115.5-py3-none-any.whl.metadata (27 kB)
  Using cached starlette-0.41.3-py3-none-any.whl.metadata (6.0 kB)
Using cached fastapi-0.115.5-py3-none-any.whl (94 kB)
Using cached starlette-0.41.3-py3-none-any.whl (73 kB)


In [24]:
!pip install uvicorn

  Using cached uvicorn-0.32.1-py3-none-any.whl.metadata (6.6 kB)
Using cached uvicorn-0.32.1-py3-none-any.whl (63 kB)



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [25]:
!pip install langserve

   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   -- ------------------------------------- 0.1/1.2 MB 1.7 MB/s eta 0:00:01
   ---- ----------------------------------- 0.1/1.2 MB 1.7 MB/s eta 0:00:01
   ------- -------------------------------- 0.2/1.2 MB 1.7 MB/s eta 0:00:01
   ----------- ---------------------------- 0.3/1.2 MB 1.9 MB/s eta 0:00:01
   ------------- -------------------------- 0.4/1.2 MB 1.9 MB/s eta 0:00:01
   ----------------- ---------------------- 0.5/1.2 MB 1.9 MB/s eta 0:00:01
   -------------------- ------------------- 0.6/1.2 MB 1.9 MB/s eta 0:00:01
   ------------------------ --------------- 0.7/1.2 MB 2.0 MB/s eta 0:00:01
   ---------------------------- ----------- 0.8/1.2 MB 2.0 MB/s eta 0:00:01
   ------------------------------- -------- 0.9/1.2 MB 2.1 MB/s eta 0:00:01
   ------------------------------------- -- 1.1/1.2 MB 2.2 MB/s eta 0:00:01
   ---------------------------------------- 1.2/1.2 MB 2.1 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [26]:
!pip install sse_starlette


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
